In [5]:
import os
from os.path import dirname, join
from shutil import copyfile
import numpy as np
import pydicom
import SimpleITK as sitk
import nibabel as nib
from nipype.interfaces import fsl
from nipype.interfaces.dcm2nii import Dcm2niix,Dcm2nii

dicom_dir = '/home/harryzhang/Desktop/DICOM_Images/'
dicom_split_dir = '/home/harryzhang/Desktop/DICOM_split/'
nifti_dir = '/home/harryzhang/Desktop/NIFTI_Images/'

In [ ]:
"""
Step 0
Read Dicom folder and store info in dictionary for future use

"""

def file_loader(patients_dir):
    """Explore patient dicom folders and store dcm filenames
    in a dictionary of patient.

    Args:
        patients_dir (str): directory that contains patients folder

    Returns:
        dictDCM (dict): patient as key, list of dicom files for each patients

    """

    # create an empty dictionary
    # for each patient, it contains list of all dicom files
    dict_dcm = {}

    for patient in os.listdir(patients_dir):
        path_dicom = os.path.join(patients_dir, patient)
        lst_files_dcm = []  # create an empty list
        for dirName, subdirList, fileList in os.walk(path_dicom):
            for filename in fileList:
                lst_files_dcm.append(os.path.join(dirName, filename))
        dict_dcm[patient] = lst_files_dcm

    return dict_dcm

#test_dict = file_loader(dicom_dir)

In [ ]:
def study_to_sequence(patient_dir, output_dir):

    for patient in os.listdir(patient_dir):
        
        pt_dir = os.path.join(patient_dir, patient)
        out_dir = os.path.join(output_dir, patient)
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        for f in os.listdir(pt_dir):
            
            ds = pydicom.dcmread(pt_dir+'/'+f)
            series_name = ds['SeriesDescription'].value.replace(" ","_").replace("<","").replace(">","")
            series_dir = os.path.join(out_dir, series_name)
            
            if not os.path.exists(series_dir):
                os.makedirs(series_dir)
            if not os.path.exists(os.path.join(series_dir, f)):
                copyfile(pt_dir+'/'+f, os.path.join(series_dir, f))


In [4]:

#study_to_sequence(dicom_dir, dicom_split_dir)

In [ ]:
# Get ref file
RefDs = pydicom.read_file(test_dict['13'][0])
RefDs

In [6]:
"""
Step 1
Dicom to Nifti format conversion

Tool used: Dcm2niix (https://github.com/rordenlab/dcm2niix)
install dcm2niix first, then use nipype wrapper instead of cmd

sample code here shows only T2, DWI and Perfusion conversion
adjust source_dir for other modalities
@param:
compression: Gz compression level, 1=fastest, 9=smallest
generated cmd line:
'dcm2niix -b y -z y -5 -x n -t n -m n -o -s n -v n source_dir' 
"""




def dcm_to_nifti(dicom_dir, nifti_dir, split = True, tool_used = 'dcm2niix'):
    
    for patient in os.listdir(dicom_dir):
        
        path_dicom = os.path.join(dicom_dir, patient)
        path_nifti = os.path.join(nifti_dir, patient)
        # make subfolder for each patient
        if not os.path.exists(path_nifti):
            os.makedirs(path_nifti)
        if split == False:
            if tool_used == 'dcm2niix':
                converter = Dcm2niix()
                converter.inputs.source_dir = path_dicom
                converter.inputs.compression = 5
                converter.inputs.merge_imgs = True
                converter.inputs.out_filename = '%d'
                converter.inputs.output_dir = path_nifti
                converter.run()
            elif tool_used == 'dcm2nii':
                converter = Dcm2nii()
                converter.inputs.source_dir = path_dicom
                converter.inputs.gzip_output = True
                converter.inputs.output_dir = path_nifti
                converter.run()
            else:
                raise Warning("tool used does not exist, please enter dcm2nii or dcm2niix")
                
        else:
            for s in os.listdir(path_dicom):
                if tool_used == 'dcm2niix':
                    converter = Dcm2niix()
                    converter.inputs.source_dir = path_dicom + '/' + s
                    converter.inputs.compression = 5
                    converter.inputs.merge_imgs = True
                    converter.inputs.out_filename = 'x_%d'
                    converter.inputs.output_dir = path_nifti
                    converter.run()
                elif tool_used == 'dcm2nii':
                    converter = Dcm2nii()
                    converter.inputs.source_dir = path_dicom + '/' + s
                    converter.inputs.gzip_output = True
                    converter.inputs.output_dir = path_nifti
                    converter.run()
                else:
                    raise Warning("tool used does not exist, please enter dcm2nii or dcm2niix")
            

    

In [7]:
dcm_to_nifti(dicom_split_dir, nifti_dir, True, 'dcm2nii')
#dcm_to_nifti(dicom_split_dir, nifti_dir, True, 'dcm2niix')   

190724-11:52:49,39 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.039460:Chris Rorden's dcm2nii :: 2MAY2016 64bit BSD License  :: (upgrade to dcm2niix suggested)
190724-11:52:49,40 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.039460:reading preferences file config.ini
190724-11:52:49,40 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.039460:Data will be exported to /home/harryzhang/Desktop/NIFTI_Images/67/
190724-11:52:49,41 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.039460:Validating 40 potential DICOM images.
190724-11:52:49,41 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.039460:Found 40 DICOM images.
190724-11:52:49,42 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.042379:Converting 1/40  volumes: 1
190724-11:52:49,42 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.042379:1.3.12.2.1107.5.2.30.26371.30000013062716164258500008073->18991230_000000TOF3DmultislabCOWs013a1001.nii
190724-11:52:49,43 nipype.interface INFO:
	 stdout 2019-07-24T11:52:4

190724-11:52:49,154 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.152817:1.3.12.2.1107.5.2.30.26371.30000013062716164258500008086->18991230_000000TOF3DmultislabCOWs013a1001M.nii
190724-11:52:49,161 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.161891: brightest voxel was 4096: data will be saved as 16-bit signed integer.
190724-11:52:49,162 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.161891:GZip...18991230_000000TOF3DmultislabCOWs013a1001M.nii.gz
190724-11:52:49,163 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.161891:Converting 15/40  volumes: 1
190724-11:52:49,164 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.161891:1.3.12.2.1107.5.2.30.26371.30000013062716164258500008087->18991230_000000TOF3DmultislabCOWs013a1001N.nii
190724-11:52:49,171 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.171814: brightest voxel was 4182: data will be saved as 16-bit signed integer.
190724-11:52:49,172 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.171814:GZip...

190724-11:52:49,306 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.306368: brightest voxel was 4096: data will be saved as 16-bit signed integer.
190724-11:52:49,307 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.306368:GZip...18991230_000000TOF3DmultislabCOWs027a1001G.nii.gz
190724-11:52:49,307 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.306368:Converting 29/40  volumes: 1
190724-11:52:49,316 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.316879:1.3.12.2.1107.5.2.30.26371.30000013062716164258500008188->18991230_000000TOF3DmultislabCOWs027a1001H.nii
190724-11:52:49,317 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.316879: brightest voxel was 4096: data will be saved as 16-bit signed integer.
190724-11:52:49,317 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.316879:GZip...18991230_000000TOF3DmultislabCOWs027a1001H.nii.gz
190724-11:52:49,318 nipype.interface INFO:
	 stdout 2019-07-24T11:52:49.316879:Converting 30/40  volumes: 1
190724-11:52:49,325 nipype

190724-11:52:50,341 nipype.interface INFO:
	 stdout 2019-07-24T11:52:50.339951:GZip...18991230_000000AxTSET2s008a1001.nii.gz
190724-11:52:50,381 nipype.interface INFO:
	 stdout 2019-07-24T11:52:50.381138:Chris Rorden's dcm2nii :: 2MAY2016 64bit BSD License  :: (upgrade to dcm2niix suggested)
190724-11:52:50,382 nipype.interface INFO:
	 stdout 2019-07-24T11:52:50.381138:reading preferences file config.ini
190724-11:52:50,382 nipype.interface INFO:
	 stdout 2019-07-24T11:52:50.381138:Data will be exported to /home/harryzhang/Desktop/NIFTI_Images/67/
190724-11:52:50,382 nipype.interface INFO:
	 stdout 2019-07-24T11:52:50.381138:Validating 26 potential DICOM images.
190724-11:52:50,382 nipype.interface INFO:
	 stdout 2019-07-24T11:52:50.381138:Found 26 DICOM images.
190724-11:52:50,419 nipype.interface INFO:
	 stdout 2019-07-24T11:52:50.419714:Converting 26/26  volumes: 1
190724-11:52:50,420 nipype.interface INFO:
	 stdout 2019-07-24T11:52:50.419714:1.3.12.2.1107.5.2.30.26371.2013062719480

190724-11:52:57,545 nipype.interface INFO:
	 stdout 2019-07-24T11:52:57.545300:Chris Rorden's dcm2nii :: 2MAY2016 64bit BSD License  :: (upgrade to dcm2niix suggested)
190724-11:52:57,546 nipype.interface INFO:
	 stdout 2019-07-24T11:52:57.545300:reading preferences file config.ini
190724-11:52:57,546 nipype.interface INFO:
	 stdout 2019-07-24T11:52:57.545300:Data will be exported to /home/harryzhang/Desktop/NIFTI_Images/67/
190724-11:52:57,546 nipype.interface INFO:
	 stdout 2019-07-24T11:52:57.545300:Validating 26 potential DICOM images.
190724-11:52:57,547 nipype.interface INFO:
	 stdout 2019-07-24T11:52:57.545300:Found 26 DICOM images.
190724-11:52:57,779 nipype.interface INFO:
	 stdout 2019-07-24T11:52:57.779219:Converting 26/26  volumes: 1
190724-11:52:57,779 nipype.interface INFO:
	 stdout 2019-07-24T11:52:57.779219:1.3.12.2.1107.5.2.30.26371.2013062719530457824703752->18991230_000000AXFLAIRs006a1001.nii
190724-11:52:57,780 nipype.interface INFO:
	 stdout 2019-07-24T11:52:57.779

...

190724-11:57:49,887 nipype.interface INFO:
	 stdout 2019-07-24T11:57:49.886382:GZip...18991230_000000Subtractions024a1001G.nii.gz
190724-11:57:49,887 nipype.interface INFO:
	 stdout 2019-07-24T11:57:49.886382:Converting 9/20  volumes: 1
190724-11:57:49,916 nipype.interface INFO:
	 stdout 2019-07-24T11:57:49.916589:1.3.12.2.1107.5.2.30.26371.30000013071720065279600000293->18991230_000000Subtractions024a1001H.nii
190724-11:57:49,917 nipype.interface INFO:
	 stdout 2019-07-24T11:57:49.916589: brightest voxel was 4165: data will be saved as 16-bit signed integer.
190724-11:57:49,917 nipype.interface INFO:
	 stdout 2019-07-24T11:57:49.916589:GZip...18991230_000000Subtractions024a1001H.nii.gz
190724-11:57:49,917 nipype.interface INFO:
	 stdout 2019-07-24T11:57:49.916589:Converting 10/20  volumes: 1
190724-11:57:49,946 nipype.interface INFO:
	 stdout 2019-07-24T11:57:49.946194:1.3.12.2.1107.5.2.30.26371.30000013071720065279600000294->18991230_000000Subtractions024a1001I.nii
190724-11:57:49,94

190724-11:57:50,227 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.226689:1.3.12.2.1107.5.2.30.26371.30000013071720065279600000168->18991230_000000TOF3DmultislabCOWs023a1001A.nii
190724-11:57:50,227 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.226689: brightest voxel was 4255: data will be saved as 16-bit signed integer.
190724-11:57:50,236 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.236348:GZip...18991230_000000TOF3DmultislabCOWs023a1001A.nii.gz
190724-11:57:50,236 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.236348:Converting 3/20  volumes: 1
190724-11:57:50,237 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.236348:1.3.12.2.1107.5.2.30.26371.30000013071720065279600000169->18991230_000000TOF3DmultislabCOWs023a1001B.nii
190724-11:57:50,237 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.236348: brightest voxel was 4264: data will be saved as 16-bit signed integer.
190724-11:57:50,246 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.246566:GZip...1

190724-11:57:50,386 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.386304: brightest voxel was 4213: data will be saved as 16-bit signed integer.
190724-11:57:50,386 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.386304:GZip...18991230_000000TOF3DmultislabCOWs023a1001O.nii.gz
190724-11:57:50,387 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.386304:Converting 17/20  volumes: 1
190724-11:57:50,387 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.386304:1.3.12.2.1107.5.2.30.26371.30000013071720065279600000183->18991230_000000TOF3DmultislabCOWs023a1001P.nii
190724-11:57:50,396 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.396440: brightest voxel was 4219: data will be saved as 16-bit signed integer.
190724-11:57:50,397 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.396440:GZip...18991230_000000TOF3DmultislabCOWs023a1001P.nii.gz
190724-11:57:50,397 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.396440:Converting 18/20  volumes: 1
190724-11:57:50,398 nipype

190724-11:57:50,542 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.542464:GZip...18991230_000000TOF3DmultislabCOWs021a1001H.nii.gz
190724-11:57:50,542 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.542464:Converting 10/20  volumes: 1
190724-11:57:50,543 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.542464:1.3.12.2.1107.5.2.30.26371.30000013071720065279600000067->18991230_000000TOF3DmultislabCOWs021a1001I.nii
190724-11:57:50,543 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.542464: brightest voxel was 4227: data will be saved as 16-bit signed integer.
190724-11:57:50,550 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.550953:GZip...18991230_000000TOF3DmultislabCOWs021a1001I.nii.gz
190724-11:57:50,551 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.550953:Converting 11/20  volumes: 1
190724-11:57:50,551 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.550953:1.3.12.2.1107.5.2.30.26371.30000013071720065279600000068->18991230_000000TOF3DmultislabCOWs021a100

190724-11:57:50,977 nipype.interface INFO:
	 stdout 2019-07-24T11:57:50.975154:Found 78 DICOM images.
190724-11:57:51,196 nipype.interface INFO:
	 stdout 2019-07-24T11:57:51.196966:Converting 78/78  volumes: 3
190724-11:57:51,197 nipype.interface INFO:
	 stdout 2019-07-24T11:57:51.196966:1.3.12.2.1107.5.2.30.26371.2013071712425466613704346->18991230_000000ep2ddiff3scantraces002a1001.nii
190724-11:57:51,197 nipype.interface INFO:
	 stdout 2019-07-24T11:57:51.196966: brightest voxel was 1967: data will be saved as 16-bit signed integer.
190724-11:57:51,198 nipype.interface INFO:
	 stdout 2019-07-24T11:57:51.196966:GZip...18991230_000000ep2ddiff3scantraces002a1001.nii.gz
190724-11:57:51,242 nipype.interface INFO:
	 stdout 2019-07-24T11:57:51.242665:Chris Rorden's dcm2nii :: 2MAY2016 64bit BSD License  :: (upgrade to dcm2niix suggested)
190724-11:57:51,243 nipype.interface INFO:
	 stdout 2019-07-24T11:57:51.242665:reading preferences file config.ini
190724-11:57:51,243 nipype.interface INF